In [1]:
%matplotlib inline
import matplotlib as mpl
plt = mpl.pyplot
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.autolimit_mode'] = 'round_numbers'
mpl.rcParams['axes.xmargin'] = 0
mpl.rcParams['axes.ymargin'] = 0
import numpy as np
import utils

In [2]:
# training_features, training_labels, validation_features, validation_labels = \
#     utils.get_training_data(0.2, onehot=False, standardize=False)
# print(training_features.shape, training_labels.shape,
#      validation_features.shape, validation_labels.shape)

In [3]:
# Tf-Idf transform
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf = TfidfTransformer()
# training_features = tfidf.fit_transform(training_features)
# validation_features = tfidf.transform(validation_features)

In [4]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Activation, Dropout

def create_model(dense=(160,), activation='sigmoid', dropout=0.7):
    model = Sequential()
    for d in dense:
        model.add(Dense(d, input_shape=(1000,)))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout))
    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
class KerasClassifierMod(KerasClassifier):
    def fit(self, x, y, sample_weight, **kwargs):
        return super().fit(x, y, sample_weight=sample_weight, **kwargs)

Using TensorFlow backend.


In [5]:
nn = KerasClassifierMod(build_fn=create_model, epochs=1, batch_size=32, verbose=1)

In [6]:
from sklearn.ensemble import AdaBoostClassifier
n_estimators = 50
learning_rate = 0.1
ada_nn = AdaBoostClassifier(base_estimator=nn, n_estimators=n_estimators, 
                        learning_rate=learning_rate)

In [7]:
# ada_nn.fit(training_features, training_labels)

In [8]:
from sklearn.metrics import zero_one_loss
def plot_ada_hist(ada, title):
    ada_acc = np.zeros((n_estimators,))
    for i, pred_labels in enumerate(ada.staged_predict(validation_features)):
        ada_acc[i] = zero_one_loss(pred_labels, validation_labels)

    ada_acc_train = np.zeros((n_estimators,))
    for i, pred_labels in enumerate(ada.staged_predict(training_features)):
        ada_acc_train[i] = zero_one_loss(pred_labels, training_labels)

    plt.figure(dpi=300)
    plt.plot(np.arange(n_estimators) + 1, ada_acc, label='AdaBoost Test Loss')
    plt.plot(np.arange(n_estimators) + 1, ada_acc_train, label='AdaBoost Train Loss')
    plt.semilogy()
    plt.legend()
    plt.title(title)
    plt.show()
    plt.clf()

In [9]:
# plot_ada_hist(ada_nn, 'AdaBoost with neural network\nfinal accuracy %.4f'
#              %ada_nn.score(validation_features, validation_labels))

In [10]:
# Now the final training
training_features1, training_labels1, _, _ = \
    utils.get_training_data(0, onehot=False, standardize=False)
print(training_features1.shape, training_labels1.shape)

(20000, 1000) (20000,)


In [11]:
test_features1 = utils.get_test_data(standardize=False)
print(test_features1.shape)

(10000, 1000)


In [12]:
# Tf-Idf transform
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
training_features1 = tfidf.fit_transform(training_features1)
test_features1 = tfidf.transform(test_features1)

In [13]:
nn = KerasClassifierMod(build_fn=create_model, epochs=2, batch_size=64, verbose=1)
from sklearn.ensemble import AdaBoostClassifier
n_estimators = 200
learning_rate = 0.05
ada_nn1 = AdaBoostClassifier(base_estimator=nn, n_estimators=n_estimators, 
                        learning_rate=learning_rate)
ada_nn1.fit(training_features1, training_labels1)

Epoch 1/2
20000/20000 [==============================] - 2s 117us/step - loss: 3.2926e-05 - acc: 0.6220
Epoch 2/2
20000/20000 [==============================] - 1s 25us/step
Epoch 1/2
20000/20000 [==============================] - 2s 124us/step - loss: 3.3580e-05 - acc: 0.6097
Epoch 2/2
20000/20000 [==============================] - 1s 27us/step
Epoch 1/2
20000/20000 [==============================] - 2s 121us/step - loss: 3.3793e-05 - acc: 0.6100
Epoch 2/2
20000/20000 [==============================] - 1s 28us/step
Epoch 1/2
20000/20000 [==============================] - 3s 125us/step - loss: 3.3435e-05 - acc: 0.6178
Epoch 2/2
20000/20000 [==============================] - 1s 30us/step
Epoch 1/2
20000/20000 [==============================] - 3s 128us/step - loss: 3.3723e-05 - acc: 0.6089
Epoch 2/2
20000/20000 [==============================] - 1s 31us/step
Epoch 1/2
20000/20000 [==============================] - 3s 132us/step - loss: 3.3707e-05 - acc: 0.6141
Epoch 2/2
20000/20000 [===

20000/20000 [==============================] - 1s 70us/step
Epoch 1/2
20000/20000 [==============================] - 4s 216us/step - loss: 3.6454e-05 - acc: 0.5831
Epoch 2/2
20000/20000 [==============================] - 1s 73us/step
Epoch 1/2
20000/20000 [==============================] - 4s 223us/step - loss: 3.6587e-05 - acc: 0.5837
Epoch 2/2
20000/20000 [==============================] - 1s 74us/step
Epoch 1/2
20000/20000 [==============================] - 5s 227us/step - loss: 3.6885e-05 - acc: 0.5778
Epoch 2/2
20000/20000 [==============================] - 2s 77us/step
Epoch 1/2
20000/20000 [==============================] - 5s 231us/step - loss: 3.6743e-05 - acc: 0.5840
Epoch 2/2
20000/20000 [==============================] - 2s 78us/step
Epoch 1/2
20000/20000 [==============================] - 5s 270us/step - loss: 3.6580e-05 - acc: 0.5866
Epoch 2/2
20000/20000 [==============================] - 2s 80us/step
Epoch 1/2
20000/20000 [==============================] - 5s 240us/step

20000/20000 [==============================] - 6s 323us/step - loss: 3.8340e-05 - acc: 0.5570
Epoch 2/2
20000/20000 [==============================] - 2s 122us/step
Epoch 1/2
20000/20000 [==============================] - 7s 327us/step - loss: 3.7966e-05 - acc: 0.5484
Epoch 2/2
20000/20000 [==============================] - 3s 125us/step
Epoch 1/2
20000/20000 [==============================] - 7s 333us/step - loss: 3.8384e-05 - acc: 0.5504
Epoch 2/2
20000/20000 [==============================] - 3s 127us/step
Epoch 1/2
20000/20000 [==============================] - 7s 334us/step - loss: 3.7915e-05 - acc: 0.5522
Epoch 2/2
20000/20000 [==============================] - 3s 129us/step
Epoch 1/2
20000/20000 [==============================] - 7s 340us/step - loss: 3.8286e-05 - acc: 0.5518
Epoch 2/2
20000/20000 [==============================] - 3s 131us/step
Epoch 1/2
20000/20000 [==============================] - 7s 342us/step - loss: 3.8415e-05 - acc: 0.5428
Epoch 2/2
20000/20000 [========

20000/20000 [==============================] - 4s 177us/step
Epoch 1/2
20000/20000 [==============================] - 9s 434us/step - loss: 3.9046e-05 - acc: 0.5297
Epoch 2/2
20000/20000 [==============================] - 4s 182us/step
Epoch 1/2
20000/20000 [==============================] - 9s 440us/step - loss: 3.9094e-05 - acc: 0.5229
Epoch 2/2
20000/20000 [==============================] - 4s 180us/step
Epoch 1/2
20000/20000 [==============================] - 9s 442us/step - loss: 3.8975e-05 - acc: 0.5221
Epoch 2/2
20000/20000 [==============================] - 4s 182us/step
Epoch 1/2
20000/20000 [==============================] - 9s 453us/step - loss: 3.9028e-05 - acc: 0.5244
Epoch 2/2
20000/20000 [==============================] - 4s 182us/step
Epoch 1/2
20000/20000 [==============================] - 9s 453us/step - loss: 3.9134e-05 - acc: 0.5259
Epoch 2/2
20000/20000 [==============================] - 4s 187us/step
Epoch 1/2
20000/20000 [==============================] - 9s 464u

20000/20000 [==============================] - 5s 240us/step
Epoch 1/2
20000/20000 [==============================] - 11s 556us/step - loss: 3.9024e-05 - acc: 0.5204
Epoch 2/2
20000/20000 [==============================] - 5s 240us/step
Epoch 1/2
20000/20000 [==============================] - 11s 564us/step - loss: 3.9336e-05 - acc: 0.5172
Epoch 2/2
20000/20000 [==============================] - 5s 240us/step
Epoch 1/2
20000/20000 [==============================] - 11s 562us/step - loss: 3.9578e-05 - acc: 0.5220
Epoch 2/2
20000/20000 [==============================] - 5s 240us/step
Epoch 1/2
20000/20000 [==============================] - 11s 565us/step - loss: 3.8969e-05 - acc: 0.5176
Epoch 2/2
20000/20000 [==============================] - 5s 245us/step
Epoch 1/2
20000/20000 [==============================] - 11s 574us/step - loss: 3.9334e-05 - acc: 0.5191
Epoch 2/2
20000/20000 [==============================] - 5s 249us/step
Epoch 1/2
20000/20000 [==============================] - 11

20000/20000 [==============================] - 14s 680us/step - loss: 3.8888e-05 - acc: 0.5107
Epoch 2/2
20000/20000 [==============================] - 6s 305us/step
Epoch 1/2
20000/20000 [==============================] - 14s 693us/step - loss: 3.9127e-05 - acc: 0.5247
Epoch 2/2
20000/20000 [==============================] - 6s 307us/step
Epoch 1/2
20000/20000 [==============================] - 14s 692us/step - loss: 3.9811e-05 - acc: 0.4960
Epoch 2/2
20000/20000 [==============================] - 6s 315us/step
Epoch 1/2
20000/20000 [==============================] - 14s 699us/step - loss: 3.9195e-05 - acc: 0.5174
Epoch 2/2
20000/20000 [==============================] - 6s 311us/step
Epoch 1/2
20000/20000 [==============================] - 14s 702us/step - loss: 3.9161e-05 - acc: 0.5093
Epoch 2/2
20000/20000 [==============================] - 6s 312us/step
Epoch 1/2
20000/20000 [==============================] - 14s 710us/step - loss: 3.9014e-05 - acc: 0.5045
Epoch 2/2
20000/20000 [==

Epoch 2/2
20000/20000 [==============================] - 8s 378us/step
Epoch 1/2
20000/20000 [==============================] - 17s 831us/step - loss: 3.8822e-05 - acc: 0.5116
Epoch 2/2
20000/20000 [==============================] - 8s 391us/step
Epoch 1/2
20000/20000 [==============================] - 17s 841us/step - loss: 3.9455e-05 - acc: 0.5051
Epoch 2/2
20000/20000 [==============================] - 8s 387us/step
Epoch 1/2
20000/20000 [==============================] - 20s 1ms/step - loss: 3.9144e-05 - acc: 0.5116
Epoch 2/2
20000/20000 [==============================] - 8s 391us/step
Epoch 1/2
20000/20000 [==============================] - 17s 834us/step - loss: 3.9397e-05 - acc: 0.5079
Epoch 2/2
20000/20000 [==============================] - 8s 380us/step
Epoch 1/2
20000/20000 [==============================] - 17s 852us/step - loss: 3.9535e-05 - acc: 0.4980
Epoch 2/2
20000/20000 [==============================] - 8s 386us/step
Epoch 1/2
20000/20000 [============================

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=<__main__.KerasClassifierMod object at 0x124e64ef0>,
          learning_rate=0.05, n_estimators=200, random_state=None)

In [14]:
test_labels1 = ada_nn1.predict(test_features1)
utils.save_prediction(test_labels1, 'data/test_labels_ada_nn.txt')

10000/10000 [==============================] - 1s 90us/step


10000/10000 [==============================] - 1s 89us/step


In [15]:
print(ada_nn1.score(training_features1, training_labels1))
# The training accuracy
# For 200 estimators and a learning rate of 0.05 it was 0.86925

20000/20000 [==============================] - 2s 89us/step


20000/20000 [==============================] - 2s 89us/step
0.8694
